# Estimating Joint Tour Composition

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

# Load data and prep model for estimation

In [3]:
modelname = "joint_tour_composition"

from activitysim.estimation.larch import component_model

model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

loading from output-est-mode/estimation_data_bundle/joint_tour_composition/joint_tour_composition_coefficients.csv
loading spec from output-est-mode/estimation_data_bundle/joint_tour_composition/joint_tour_composition_SPEC.csv
loading from output-est-mode/estimation_data_bundle/joint_tour_composition/joint_tour_composition_values_combined.parquet


# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

## Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coef_unavailable,-999.0000,T
coef_asc_children,5.3517,F
coef_asc_mixed,5.6290,fF
coef_tour_purpose_is_eating_out_children,-0.9678,F
coef_tour_purpose_is_eating_out_mixed,-0.8027,F
coef_tour_purpose_is_discretionary_adults,0.7648,F
coef_tour_purpose_is_discretionary_children,0.5101,F
coef_number_of_full_time_workers_adults,1.0240,F
coef_number_of_full_time_workers_mixed,0.3624,F


## Utility specification

In [5]:
data.spec

,Label,Description,Expression,adults,children,mixed
0,util_asc,Alternative-specific constant,1,NaN,coef_asc_children,coef_asc_mixed
1,util_tour_purpose_is_eating_out,Joint tour purpose is eating out (dummy),tour_type=='eat',NaN,coef_tour_purpose_is_eating_out_children,coef_tour_purpose_is_eating_out_mixed
2,util_tour_purpose_is_discretionary,Joint tour purpose is discretionary (dummy),tour_type=='disc',coef_tour_purpose_is_discretionary_adults,coef_tour_purpose_is_discretionary_children,NaN
3,util_number_of_full_time_workers,Number of Full-Time Workers in the household,num_full_max3,coef_number_of_full_time_workers_adults,NaN,coef_number_of_full_time_workers_mixed
4,util_number_of_part_time_workers,Number of Part-Time Workers in the household,num_part_max3,coef_number_of_part_time_workers_adults,NaN,coef_number_of_part_time_workers_mixed
5,util_number_of_university_students,Number of University students in the household,num_univ_max3,coef_number_of_university_students,NaN,NaN
6,util_number_of_non_workers,Number of Non-Workers in the household,num_nonwork_max3,coef_number_of_non_workers_adults,NaN,coef_number_of_non_workers_mixed
7,util_number_of_children_too_young_for_school,Number of Children too Young for School in the...,num_preschool_max3,NaN,coef_number_of_children_too_young_for_school_c...,coef_number_of_children_too_young_for_school_m...
8,util_number_of_pre_driving_age_children,Number of Pre-driving Age Children in the hous...,num_school_max3,NaN,coef_number_of_pre_driving_age_children_children,coef_number_of_pre_driving_age_children_mixed
9,util_number_of_driving_age_children,Number of Driving-age Children in the household,num_driving_max3,NaN,coef_number_of_driving_age_children_children,coef_number_of_driving_age_children_mixed


## Chooser data

In [6]:
data.chooser_data

,tour_id,model_choice,override_choice,util_asc,util_tour_purpose_is_eating_out,util_tour_purpose_is_discretionary,util_number_of_full_time_workers,util_number_of_part_time_workers,util_number_of_university_students,util_number_of_non_workers,...,log_time_window_overlap_adult_child,num_full_max3,num_part_max3,num_univ_max3,num_nonwork_max3,num_preschool_max3,num_school_max3,num_driving_max3,more_cars_than_workers,override_choice_code
household_id,,,,,,,,,,,,,,,,,,,,,
190592,7853686,adults,adults,1.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.000000,0.0,0.0,0.0,2.0,0.0,0.0,0.0,True,1
191185,7902318,adults,adults,1.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.000000,0.0,0.0,0.0,2.0,0.0,0.0,0.0,True,1
191276,7909776,adults,adults,1.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.000000,0.0,0.0,0.0,2.0,0.0,0.0,0.0,True,1
192239,7988742,adults,adults,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,True,1
193643,8103874,adults,adults,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753390,303771147,mixed,mixed,1.0,0.0,0.0,3.0,0.0,0.0,0.0,...,2.708984,3.0,0.0,0.0,0.0,2.0,0.0,0.0,True,3
2754817,304144212,mixed,mixed,1.0,0.0,0.0,3.0,0.0,0.0,2.0,...,2.996094,3.0,0.0,0.0,2.0,2.0,0.0,0.0,False,3
2756049,304463637,mixed,mixed,1.0,0.0,0.0,3.0,0.0,2.0,0.0,...,2.484375,3.0,0.0,2.0,0.0,0.0,1.0,0.0,True,3


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [7]:
model.estimate(method='SLSQP')

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_asc_children,6.744041,6.744041,5.3517,-50.0,50.0,0.0,0
coef_asc_mixed,6.886657,6.886657,5.6290,-50.0,50.0,0.0,0
coef_household_has_more_cars_than_workers_adults,3.138920,3.138920,1.3860,-50.0,50.0,0.0,0
coef_household_has_more_cars_than_workers_mixed,0.384250,0.384250,0.7510,-50.0,50.0,0.0,0
coef_household_in_suburban_area_adults,-1.123570,-1.123570,0.5105,-50.0,50.0,0.0,0
coef_household_in_suburban_area_mixed,0.258299,0.258299,0.1283,-50.0,50.0,0.0,0
coef_household_in_urban_area,-0.441828,-0.441828,0.5741,-50.0,50.0,0.0,0
coef_log_max_overlap_of_adults_time_windows,1.288269,1.288269,1.1920,-50.0,50.0,0.0,0
coef_log_max_overlap_of_childrens_time_windows,1.783892,1.783892,1.8410,-50.0,50.0,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/jaxmodel.py:1156: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  self.calculate_parameter_covariance()


┣                x: coef_asc_children                                          6.744041
┃                   coef_asc_mixed                                             6.886657
┃                   coef_household_has_more_cars_than_workers_adults           3.138920
┃                   coef_household_has_more_cars_than_workers_mixed            0.384250
┃                   coef_household_in_suburban_area_adults                    -1.123570
┃                   coef_household_in_suburban_area_mixed                      0.258299
┃                   coef_household_in_urban_area                              -0.441828
┃                   coef_log_max_overlap_of_adults_time_windows                1.288269
┃                   coef_log_max_overlap_of_childrens_time_windows             1.783892
┃                   coef_log_max_overlap_of_time_windows                       1.505521
┃                   coef_low_income_households_adults                          2.520875
┃                   coef_low_income_households_mixed                           0.683195
┃                   coef_medium_income_households                              1.853225
┃                   coef_number_of_children_too_young_for_school_children      0.661222
┃                   coef_number_of_children_too_young_for_school_mixed         1.065525
┃                   coef_number_of_driving_age_children_children               0.646806
┃                   coef_number_of_driving_age_children_mixed                  0.494992
┃                   coef_number_of_full_time_workers_adults                    1.638383
┃                   coef_number_of_full_time_workers_mixed                     0.106573
┃                   coef_number_of_non_workers_adults                          0.344889
┃                   coef_number_of_non_workers_mixed                          -0.067851
┃                   coef_number_of_part_time_workers_adults                    0.094715
┃                   coef_number_of_part_time_workers_mixed                     0.182067
┃                   coef_number_of_pre_driving_age_children_children           0.256545
┃                   coef_number_of_pre_driving_age_children_mixed              0.311384
┃                   coef_number_of_university_students                         0.067485
┃                   coef_tour_purpose_is_discretionary_adults                  0.764800
┃                   coef_tour_purpose_is_discretionary_children                0.510100
┃                   coef_tour_purpose_is_eating_out_children                  -0.967800
┃                   coef_tour_purpose_is_eating_out_mixed                     -0.802700
┃                   coef_unavailable                                        -999.000000
┃                   dtype: float64
┣          logloss: 0.2061557647817926
┣        d_logloss: coef_asc_children                                        4.680292e-05
┃                   coef_asc_mixed                                           1.506794e-04
┃                   coef_household_has_more_cars_than_workers_adults        -1.303235e-04
┃                   coef_household_has_more_cars_than_workers_mixed          4.087211e-05
┃                   coef_household_in_suburban_area_adults                  -2.788428e-05
┃                   coef_household_in_suburban_area_mixed                   -2.733559e-05
┃                   coef_household_in_urban_area                            -1.695981e-04
┃                   coef_log_max_overlap_of_adults_time_windows              7.673766e-05
┃                   coef_log_max_overlap_of_childrens_time_windows           3.533351e-05
┃                   coef_log_max_overlap_of_time_windows                     2.703275e-05
┃                   coef_low_income_households_adults                       -1.364236e-04
┃                   coef_low_income_households_mixed                         5.443801e-05
┃                   coef_medium_income_households                           -7.972989e-05
┃                   coef_num

*Note that in the example data for this model, there are only 91 joint tours, which an is insufficient
number of observations to successfully estimate all 31 parameters in this model.*

### Estimated coefficients

In [8]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
Parameter,,,,,,
coef_asc_children,6.74,NA,NA,,0.00,
coef_asc_mixed,6.89,NA,NA,,0.00,
coef_household_has_more_cars_than_workers_adults,3.14,1.19,2.64,**,0.00,
coef_household_has_more_cars_than_workers_mixed,0.384,0.275,1.40,,0.00,
coef_household_in_suburban_area_adults,-1.12,NA,NA,,0.00,
coef_household_in_suburban_area_mixed,0.258,0.272,0.95,,0.00,
coef_household_in_urban_area,-0.442,NA,NA,,0.00,
coef_log_max_overlap_of_adults_time_windows,1.29,0.921,1.40,,0.00,
coef_log_max_overlap_of_childrens_time_windows,1.78,0.379,4.71,***,0.00,


# Output Estimation Results

In [9]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [10]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [11]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_unavailable,-999.000000,T
1,coef_asc_children,6.744041,F
2,coef_asc_mixed,6.886657,fF
3,coef_tour_purpose_is_eating_out_children,-0.967800,F
4,coef_tour_purpose_is_eating_out_mixed,-0.802700,F
5,coef_tour_purpose_is_discretionary_adults,0.764800,F
6,coef_tour_purpose_is_discretionary_children,0.510100,F
7,coef_number_of_full_time_workers_adults,1.638383,F
8,coef_number_of_full_time_workers_mixed,0.106573,F
9,coef_number_of_part_time_workers_adults,0.094715,F
